# Sekwencyjny model CNN z Keras i własną Bazą Danych
W tym zeszycie używam własnej, przygotowanej wcześniej bazy obrazków do analizy.
Obrazki mogą być w dowolnym formacie, ale z przyzwyczajenia użyłem takiego samego jak w MNIST.
Tym razem jednak są w formacie PNG (mogą być i JPG), w skali szarości i ich procesowanie wyglądało nieco inaczej.
Jak przygotować zbiór danych piszę poniżej:

<Tu wstaw opis jak przygotowałeś zbiór danych jak już kod będzie działał poprawnie>

In [ ]:
# Poniższy model daje nam jako takie pojęcie o tym w jaki sposób można przygotować niewielką ilość danych tak,
# aby model jednak się czegoś nauczył. W końcu nie zawsze mamy jakieś pół miliona rekordów. Ja dostałem powalającą
# ilość 150 case'ów ćwiczebnych, także trzeba było kombinować. Poniżej masz rezultat mojego bezczelnego zżyniania,
# optymalizowania i kombinowania. Enjoy. :P

# Oryginalny kod pożyczony w słusznej sprawie z: https://gist.github.com/fchollet/0830affa1f7f19fd47b06d4cf89ed44d

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# Podajemy wymiary obrazka. Ja używam formatu takiego jak w MNIST, ale możesz mieć każdy inny...
img_width, img_height = 28, 28

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 120
nb_validation_samples = 30
epochs = 50
batch_size = 5
color_cha = 1 # Ilość kanałów. W Odcieniach szarości będziesz miał 1, W kolorze (RGB) będziesz miał 3.

# Ten fragment odczytuje z obrazków w jakim formacie są zapisane kanały kodowania kolorów.
if K.image_data_format() == 'channels_first':
    input_shape = (color_cha, img_width, img_height)
else:
    input_shape = (img_width, img_height, color_cha)

# Część główna modelu. W porównaniu do TF, Keras jest bardzo prosty. 
# Jak klocki lego układa się warstwa na warstwie.
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Konfiguracja preprocesująca, której używamy do treningu.
train_datagen = ImageDataGenerator(
# obowiązkowe przeskalowanie z obowiązkowej skali int 0-225 na float32 0-1
    rescale=1. / 255,
    # Przycięcie obrazka do wartości zbliżenia.
    shear_range=0.2,
    # Powiększenie obrazka o wartość przycięcia.
    zoom_range=0.2,)
# Jakbyśmy mieli np. obrazki psów i kotów, albo samochodów, to można je jeszcze np. przerzucić w pionie. 
# W wypadku pewnych zbiorów danych jak np. litery, cyfry itp. znaki, to jest imho wątpliwe ;)
# Kod przerzucający poniżej: horizontal_flip=True. Wrzuć wraz z resztą w nawias.

# Konfiguracja preprocesująca, której użyjemy do testów:
# Tylko przeskalowanie nasycenia z int na float.
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')